In [1]:
# Testing the availability of a GPU
from theano import function, config, shared, sandbox
import theano.tensor as T
import numpy
import time

vlen = 10 * 30 * 768  # 10 x #cores x # threads per core
iters = 1000

rng = numpy.random.RandomState(22)
x = shared(numpy.asarray(rng.rand(vlen), config.floatX))
f = function([], T.exp(x))
print(f.maker.fgraph.toposort())
t0 = time.time()
for i in range(iters):
    r = f()
t1 = time.time()
print("Looping %d times took %f seconds" % (iters, t1 - t0))
print("Result is %s" % (r,))
if numpy.any([isinstance(x.op, T.Elemwise) for x in f.maker.fgraph.toposort()]):
    print('Used the cpu')
else:
    print('Used the gpu')

DEBUG: nvcc STDOUT mod.cu
   Creating library C:/Users/nikh/AppData/Local/Theano/compiledir_Windows-10-10.0.10586-Intel64_Family_6_Model_69_Stepping_1_GenuineIntel-3.4.4-64/cuda_ndarray/cuda_ndarray.lib and object C:/Users/nikh/AppData/Local/Theano/compiledir_Windows-10-10.0.10586-Intel64_Family_6_Model_69_Stepping_1_GenuineIntel-3.4.4-64/cuda_ndarray/cuda_ndarray.exp

Using gpu device 0: GeForce 820M (CNMeM is disabled, CuDNN not available)
DEBUG: nvcc STDOUT mod.cu
   Creating library C:/Users/nikh/AppData/Local/Theano/compiledir_Windows-10-10.0.10586-Intel64_Family_6_Model_69_Stepping_1_GenuineIntel-3.4.4-64/tmp0vhdau5o/meb404c8cd39208f6884dd773b584b7d7.lib and object C:/Users/nikh/AppData/Local/Theano/compiledir_Windows-10-10.0.10586-Intel64_Family_6_Model_69_Stepping_1_GenuineIntel-3.4.4-64/tmp0vhdau5o/meb404c8cd39208f6884dd773b584b7d7.exp

DEBUG: nvcc STDOUT mod.cu
   Creating library C:/Users/nikh/AppData/Local/Theano/compiledir_Windows-10-10.0.10586-Intel64_Family_6_Model_69_St

[GpuElemwise{exp,no_inplace}(<CudaNdarrayType(float32, vector)>), HostFromGpu(GpuElemwise{exp,no_inplace}.0)]
Looping 1000 times took 3.168245 seconds
Result is [ 1.23178029  1.61879349  1.52278066 ...,  2.20771813  2.29967761
  1.62323296]
Used the gpu


In [4]:
cd "E:\UCSC\218\code\python3-version-mnielsen-neural-networks-and-deep-learning\src"

E:\UCSC\218\code\python3-version-mnielsen-neural-networks-and-deep-learning\src


In [5]:
import network3
from network3 import Network
from network3 import ConvPoolLayer, FullyConnectedLayer, SoftmaxLayer
import time

In [3]:
training_data, validation_data, test_data = network3.load_data_shared()
mini_batch_size = 10

'''
To get a baseline, we'll start with a shallow architecture using just a single hidden layer, 
containing 100 hidden neurons. We'll train for 60epochs,
using a learning rate of η=0.1, a mini-batch size of 10, and no regularization.
'''
net = Network([
        FullyConnectedLayer(n_in=784, n_out=100),
        SoftmaxLayer(n_in=100, n_out=10)], mini_batch_size)

print("####################BASELINE############################")

t0 = time.time()
net.SGD(training_data, 60, mini_batch_size, 0.1, 
            validation_data, test_data)
t1 = time.time()

print(t1-t0)
'''
Epoch 59: validation accuracy 97.72%
Finished training network.
Best validation accuracy of 97.73% obtained at iteration 184999
Corresponding test accuracy of 97.89%
751.523999929 seconds
"I obtained a best classification accuracy of 97.80 percent." - M Nielsen
'''

'''
Let's begin by inserting a convolutional layer, right at the beginning of the network. 
We'll use 5 by 5 local receptive fields, a stride length of 1, 
and 20 feature maps. We'll also insert a max-pooling layer, 
which combines the features using 2 by 2 pooling windows.
Let's train such a network, and see how it performs'''
net = Network([
    ConvPoolLayer(image_shape=(mini_batch_size, 1, 28, 28), 
                  filter_shape=(20, 1, 5, 5), 
                  poolsize=(2, 2)),
    FullyConnectedLayer(n_in=20*12*12, n_out=100),
    SoftmaxLayer(n_in=100, n_out=10)], mini_batch_size)

print("####################1 CONV LAYER############################")

t0 = time.time()
net.SGD(training_data, 60, mini_batch_size, 0.1, 
            validation_data, test_data)   
t1 = time.time()
print(t1-t0)
'''
Epoch 59: validation accuracy 98.79%
This is the best validation accuracy to date.
The corresponding test accuracy is 98.69%
Finished training network.
Best validation accuracy of 98.79% obtained at iteration 299999
Corresponding test accuracy of 98.69%
8984.14199996 seconds
"That gets us to 98.78 percent accuracy" - M. Nielsen
'''

'''
Let's try inserting a second convolutional-pooling layer. 
We'll make the insertion between the existing convolutional-pooling layer 
and the fully-connected hidden layer. 
Again, we'll use a 5×5 local receptive field, and pool over 2×2 regions.
'''
net = Network([
    ConvPoolLayer(image_shape=(mini_batch_size, 1, 28, 28), 
                  filter_shape=(20, 1, 5, 5), 
                  poolsize=(2, 2)),
    ConvPoolLayer(image_shape=(mini_batch_size, 20, 12, 12), 
                  filter_shape=(40, 20, 5, 5), 
                  poolsize=(2, 2)),
    FullyConnectedLayer(n_in=40*4*4, n_out=100),
    SoftmaxLayer(n_in=100, n_out=10)], mini_batch_size)

print("####################2 CONV LAYER############################")

t0 = time.time()
net.SGD(training_data, 60, mini_batch_size, 0.1, 
            validation_data, test_data) 
t1 = time.time()

print(t1-t0)


####################BASELINE############################


DEBUG: nvcc STDOUT mod.cu
   Creating library C:/Users/nikh/AppData/Local/Theano/compiledir_Windows-10-10.0.10586-Intel64_Family_6_Model_69_Stepping_1_GenuineIntel-3.4.4-64/tmpem5alw28/mc12b8f0fe265b1ada8bce924bc0f113d.lib and object C:/Users/nikh/AppData/Local/Theano/compiledir_Windows-10-10.0.10586-Intel64_Family_6_Model_69_Stepping_1_GenuineIntel-3.4.4-64/tmpem5alw28/mc12b8f0fe265b1ada8bce924bc0f113d.exp

DEBUG: nvcc STDOUT mod.cu
   Creating library C:/Users/nikh/AppData/Local/Theano/compiledir_Windows-10-10.0.10586-Intel64_Family_6_Model_69_Stepping_1_GenuineIntel-3.4.4-64/tmp3ju56i7m/m0febfb49a405950dd5c14b1aff05e2db.lib and object C:/Users/nikh/AppData/Local/Theano/compiledir_Windows-10-10.0.10586-Intel64_Family_6_Model_69_Stepping_1_GenuineIntel-3.4.4-64/tmp3ju56i7m/m0febfb49a405950dd5c14b1aff05e2db.exp

DEBUG: nvcc STDOUT mod.cu
   Creating library C:/Users/nikh/AppData/Local/Theano/compiledir_Windows-10-10.0.10586-Intel64_Family_6_Model_69_Stepping_1_GenuineIntel-3.4.4-64/tmp

Training mini-batch number 0
Training mini-batch number 1000
Training mini-batch number 2000
Training mini-batch number 3000
Training mini-batch number 4000
Epoch 0: validation accuracy 92.64%
This is the best validation accuracy to date.
The corresponding test accuracy is 91.79%
Training mini-batch number 5000
Training mini-batch number 6000
Training mini-batch number 7000
Training mini-batch number 8000
Training mini-batch number 9000
Epoch 1: validation accuracy 94.53%
This is the best validation accuracy to date.
The corresponding test accuracy is 94.06%
Training mini-batch number 10000
Training mini-batch number 11000
Training mini-batch number 12000
Training mini-batch number 13000
Training mini-batch number 14000
Epoch 2: validation accuracy 95.53%
This is the best validation accuracy to date.
The corresponding test accuracy is 95.29%
Training mini-batch number 15000
Training mini-batch number 16000
Training mini-batch number 17000
Training mini-batch number 18000
Training mini-

DEBUG: nvcc STDOUT mod.cu
   Creating library C:/Users/nikh/AppData/Local/Theano/compiledir_Windows-10-10.0.10586-Intel64_Family_6_Model_69_Stepping_1_GenuineIntel-3.4.4-64/tmp2pjoh958/m7e0ff767eb2070995a03f61623decf87.lib and object C:/Users/nikh/AppData/Local/Theano/compiledir_Windows-10-10.0.10586-Intel64_Family_6_Model_69_Stepping_1_GenuineIntel-3.4.4-64/tmp2pjoh958/m7e0ff767eb2070995a03f61623decf87.exp

DEBUG: nvcc STDOUT mod.cu
   Creating library C:/Users/nikh/AppData/Local/Theano/compiledir_Windows-10-10.0.10586-Intel64_Family_6_Model_69_Stepping_1_GenuineIntel-3.4.4-64/tmp7yqm7hz2/m4502c9fd601ea77a4b6e499475065fe1.lib and object C:/Users/nikh/AppData/Local/Theano/compiledir_Windows-10-10.0.10586-Intel64_Family_6_Model_69_Stepping_1_GenuineIntel-3.4.4-64/tmp7yqm7hz2/m4502c9fd601ea77a4b6e499475065fe1.exp

DEBUG: nvcc STDOUT mod.cu
   Creating library C:/Users/nikh/AppData/Local/Theano/compiledir_Windows-10-10.0.10586-Intel64_Family_6_Model_69_Stepping_1_GenuineIntel-3.4.4-64/tmp

Training mini-batch number 0
Training mini-batch number 1000
Training mini-batch number 2000
Training mini-batch number 3000
Training mini-batch number 4000
Epoch 0: validation accuracy 93.33%
This is the best validation accuracy to date.
The corresponding test accuracy is 92.60%
Training mini-batch number 5000
Training mini-batch number 6000
Training mini-batch number 7000
Training mini-batch number 8000
Training mini-batch number 9000
Epoch 1: validation accuracy 95.84%
This is the best validation accuracy to date.
The corresponding test accuracy is 95.15%
Training mini-batch number 10000
Training mini-batch number 11000
Training mini-batch number 12000
Training mini-batch number 13000
Training mini-batch number 14000
Epoch 2: validation accuracy 96.81%
This is the best validation accuracy to date.
The corresponding test accuracy is 96.39%
Training mini-batch number 15000
Training mini-batch number 16000
Training mini-batch number 17000
Training mini-batch number 18000
Training mini-

DEBUG: nvcc STDOUT mod.cu
   Creating library C:/Users/nikh/AppData/Local/Theano/compiledir_Windows-10-10.0.10586-Intel64_Family_6_Model_69_Stepping_1_GenuineIntel-3.4.4-64/tmp55guaojd/mc7fe585c46cc4cfb2efe4143e401695a.lib and object C:/Users/nikh/AppData/Local/Theano/compiledir_Windows-10-10.0.10586-Intel64_Family_6_Model_69_Stepping_1_GenuineIntel-3.4.4-64/tmp55guaojd/mc7fe585c46cc4cfb2efe4143e401695a.exp

DEBUG: nvcc STDOUT mod.cu
   Creating library C:/Users/nikh/AppData/Local/Theano/compiledir_Windows-10-10.0.10586-Intel64_Family_6_Model_69_Stepping_1_GenuineIntel-3.4.4-64/tmpd6u43__q/m38ca21096e09570f6f9ea7783a6f5e60.lib and object C:/Users/nikh/AppData/Local/Theano/compiledir_Windows-10-10.0.10586-Intel64_Family_6_Model_69_Stepping_1_GenuineIntel-3.4.4-64/tmpd6u43__q/m38ca21096e09570f6f9ea7783a6f5e60.exp

DEBUG: nvcc STDOUT mod.cu
   Creating library C:/Users/nikh/AppData/Local/Theano/compiledir_Windows-10-10.0.10586-Intel64_Family_6_Model_69_Stepping_1_GenuineIntel-3.4.4-64/tmp

Training mini-batch number 0
Training mini-batch number 1000
Training mini-batch number 2000
Training mini-batch number 3000
Training mini-batch number 4000
Epoch 0: validation accuracy 91.60%
This is the best validation accuracy to date.
The corresponding test accuracy is 91.51%
Training mini-batch number 5000
Training mini-batch number 6000
Training mini-batch number 7000
Training mini-batch number 8000
Training mini-batch number 9000
Epoch 1: validation accuracy 96.64%
This is the best validation accuracy to date.
The corresponding test accuracy is 96.33%
Training mini-batch number 10000
Training mini-batch number 11000
Training mini-batch number 12000
Training mini-batch number 13000
Training mini-batch number 14000
Epoch 2: validation accuracy 97.57%
This is the best validation accuracy to date.
The corresponding test accuracy is 97.22%
Training mini-batch number 15000
Training mini-batch number 16000
Training mini-batch number 17000
Training mini-batch number 18000
Training mini-

In [7]:
training_data, validation_data, test_data = network3.load_data_shared()
mini_batch_size = 10
net = Network([
    ConvPoolLayer(image_shape=(mini_batch_size, 1, 28, 28), 
                  filter_shape=(20, 1, 5, 5), 
                  poolsize=(2, 2)),
    SoftmaxLayer(n_in=20*12*12, n_out=10)], mini_batch_size)

print("####################1 CONV LAYER WITHOUT FULLY CONNECTED########################")

t0 = time.time()
net.SGD(training_data, 60, mini_batch_size, 0.1, 
            validation_data, test_data)   
t1 = time.time()
print(t1-t0)

####################1 CONV LAYER WITHOUT FULLY CONNECTED########################
Training mini-batch number 0
Training mini-batch number 1000
Training mini-batch number 2000
Training mini-batch number 3000
Training mini-batch number 4000
Epoch 0: validation accuracy 93.44%
This is the best validation accuracy to date.
The corresponding test accuracy is 92.65%
Training mini-batch number 5000
Training mini-batch number 6000
Training mini-batch number 7000
Training mini-batch number 8000
Training mini-batch number 9000
Epoch 1: validation accuracy 96.40%
This is the best validation accuracy to date.
The corresponding test accuracy is 96.08%
Training mini-batch number 10000
Training mini-batch number 11000
Training mini-batch number 12000
Training mini-batch number 13000
Training mini-batch number 14000
Epoch 2: validation accuracy 97.54%
This is the best validation accuracy to date.
The corresponding test accuracy is 97.26%
Training mini-batch number 15000
Training mini-batch number 16000

In [ ]:
# HW1:
# Comparing NNs with 1 Convolutional layer with vs without a fully connected layer

# WITHOUT a fully connected layer
#Best validation accuracy of 98.59% obtained at iteration 264999
#Corresponding test accuracy of 98.44%

# WITH a fully connected layer
#Best validation accuracy of 98.73% obtained at iteration 299999
#Corresponding test accuracy of 98.64%